In [1]:
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy
import cv2
# Gym is an OpenAI toolkit for RL
import gym
from gym.wrappers import FrameStack, GrayScaleObservation, TransformObservation

# NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace

# Super Mario environment for OpenAI Gym
import gym_super_mario_bros
from gym.spaces import Box
from gym import Wrapper
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY
import subprocess as sp
from skimage import transform
from tqdm import tqdm

import os
os.environ['OMP_NUM_THREADS'] = '1'
import argparse
import torch
from env import create_train_env
from model import ActorCritic
from optimizer import GlobalAdam
from a3c_process import local_train, local_test
import torch.multiprocessing as _mp
import shutil

In [2]:
class opt:
    def __init__(self):
        self.world = 1
        self.stage = 1
        self.action_type = 'complex'
        self.lr = 1e-4
        self.gamma = 0.9
        self.tau = 1.0
        self.beta = 0.01
        self.num_local_steps = 50
        self.num_global_steps = 5e6
        self.num_processes = 12
        self.save_interval = 500
        self.max_actions = 200
        self.log_path = "tensorboard/a3c_super_mario_bros"
        self.saved_path = "trained_models"
        self.load_from_previous_stage = False
        self.use_gpu = True

In [3]:
opt = opt()

In [4]:
opt.log_path

'tensorboard/a3c_super_mario_bros'

In [5]:
def train(opt):
    torch.manual_seed(123)
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
        
    os.makedirs(opt.log_path)
    
    if not os.path.isdir(opt.saved_path):
        os.makedirs(opt.saved_path)
    
    mp = _mp.get_context("spawn")
    env, num_states, num_actions = create_train_env(opt.world, opt.stage, opt.action_type)
    
    global_model = ActorCritic(num_states, num_actions)
    
    if opt.use_gpu:
        global_model.cuda()
    global_model.share_memory()
    if opt.load_from_previous_stage:
        if opt.stage == 1:
            previous_world = opt.world - 1
            previous_stage = 4
        else:
            previous_world = opt.world
            previous_stage = opt.stage - 1
        file_ = "{}/a3c_super_mario_bros_{}_{}".format(opt.saved_path, previous_world, previous_stage)
        if os.path.isfile(file_):
            global_model.load_state_dict(torch.load(file_))

    optimizer = GlobalAdam(global_model.parameters(), lr=opt.lr)
    processes = []
    for index in range(opt.num_processes):
        if index == 0:
            process = mp.Process(target=local_train, args=(index, opt, global_model, optimizer, True))
        else:
            process = mp.Process(target=local_train, args=(index, opt, global_model, optimizer))
        process.start()
        processes.append(process)
    process = mp.Process(target=local_test, args=(opt.num_processes, opt, global_model))
    process.start()
    processes.append(process)
    for process in processes:
        process.join()

In [6]:
train(opt)

/home/sagarnildass/anaconda3/envs/mlagents_unity/lib/python3.9/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3` with the environment ID `SuperMarioBros-1-1-v3`.
  logger.warn(


PicklingError: Can't pickle <class '__main__.opt'>: it's not the same object as __main__.opt